In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import random
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import pickle
pd.set_option('display.max_columns', 1000)  # or 1000
pd.set_option('display.max_rows', 1000)  # or 1000
import shap
import time
from numpy import concatenate
Scaler = MinMaxScaler()

In [3]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import Dropout

# Specify the path and folder

In [373]:
infile_path = 'D:\PDF_Uppsala\Training dataset\Process-based model_Jorrit'
trainning_folder = 'D:\PDF_Uppsala\Training dataset'
LSTM_save_folder = 'D:\\PDF_Uppsala\\Work record\\3-Hydrodynamics from PM scenario\\Data frequency sensitivity test\\LSTM\\2-testing leave one year out'

In [334]:
# convert series to supervised learning
def series_to_supervised(data, hyperparameters, var_name,dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1] # number of variables
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(hyperparameters['time_steps'], 0, -1):
        cols.append(df.shift(i))
        names += [(var_name[j]+'(t-%d)' % (i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, hyperparameters['n_out']):
        cols.append(df.shift(-i))
        if i == 0:
            names += [(var_name[j]+'(t)') for j in range(n_vars)]
        else:
            names += [(var_name[j]+'(t+%d)' % (i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    return agg

In [118]:
# load dataset
def load_dataset(df,var_name):
    values = df[var_name].values
    return values

In [354]:
# reframe dataset
def reframe(values,hyperparameters,var_names):
    reframed = series_to_supervised(values, hyperparameters,var_names)
    reframed = reframed.iloc[hyperparameters['time_steps']:]
    drop_col =[]
    for i in range(1,hyperparameters['time_steps']+1):
        drop_col += [len(var_name)*i-1]
    reframed.drop(reframed.iloc[:,drop_col],axis=1,inplace = True)
    return reframed

In [120]:
def sparse_dataset(data_X,data_y):
    index = []
    y = []
    for i in range(len(data_y)):
        if ~np.isnan(data_y[i]):
            index.append(i)
            y.append(data_y[i])
    X = np.stack(data_X[index,:,:])
    y = np.array(y)
    return index,X,y

In [121]:
def fit_lstm(train_X,train_y,n_batch,nb_epoch,n_neuros,dropout,verbose):
    # design network
    model = Sequential()
    model.add(LSTM(n_neuros,  return_sequences = True,
              input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dropout(dropout))
    model.add(LSTM(n_neuros, return_sequences = True))
    model.add(Dropout(dropout))
    model.add(LSTM(n_neuros))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    # fit network
    model.fit(train_X,train_y,epochs =nb_epoch,batch_size = n_batch,verbose = verbose)
    return model

In [123]:
def split_dataset(train,test,time_steps):
    # split into input and outputs
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]
    # reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], time_steps+1, int(train_X.shape[1]/(time_steps+1))))
    test_X = test_X.reshape((test_X.shape[0], time_steps+1, int(test_X.shape[1]/(time_steps+1))))
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    print('number of input timesteps: {}'.format(train_X.shape[1]))
    print('number of features: {}'.format(train_X.shape[2]))
    return train_X, train_y,test_X, test_y

In [336]:
# ensure all data is float
def predict_lstm(Nut_memory,test_yr,var_name,nutrient,hyperparameters):
    values = load_dataset(Nut_memory[Nut_memory['year']!=test_yr],var_name) #values = values.astype('float32')
    # frame as supervised learning
    n_var = len(var_name)
    reframed = reframe(values,hyperparameters,var_name)
    train = reframed.values
    values = load_dataset(Nut_memory[Nut_memory['year']==test_yr],var_name) #values = values.astype('float32')
    reframed = reframe(values,hyperparameters,var_name)
    test = reframed.values
    train_X, train_y,test_X, test_y = split_dataset(train,test,hyperparameters['time_steps'])
    # fit the lstm model
    index,X,y = sparse_dataset(train_X,train_y) # stack the timeseries input together to create a 2D training input X, and a 1D lable y
    y_scaled = Scaler.fit_transform(y.reshape(-1,1))
    print('number of samples: {}'.format(len(index)))
    model = fit_lstm(X,y_scaled,
                     hyperparameters['n_batch'],hyperparameters['nb_epoch'],
                     hyperparameters['n_neuros'],hyperparameters['dropout'],hyperparameters['verbose'])
    yhat = Scaler.inverse_transform(model.predict(test_X,batch_size = hyperparameters['n_batch']))
    x = np.empty((hyperparameters['time_steps'],1))
    x[:] = np.nan
    yhat = np.concatenate((x,yhat))
    df = pd.DataFrame({'Date':Nut_memory[Nut_memory['year']==test_yr]['Date'],nutrient:yhat.flatten()})
    return df,model,yhat

In [183]:
def compare(Erken_Nut,Nut_memory,nutrient,RMSE,R2,hat):
    compare = Erken_Nut.merge(Nut_memory,on = 'Date',how = 'left')[['Date',nutrient+'_x',nutrient+'_y']].dropna()
    compare.columns = [['Date','ML','OB']]
    rmse = sqrt(mean_squared_error(compare['OB'], compare['ML']))
    r2 = r2_score(compare['OB'], compare['ML'])
    RMSE.append(rmse)
    R2.append(r2)
    # Add the time-series prediction into sample dataset for next variable modeling
    Nut_memory.loc[Nut_memory['year']==test_yr,nutrient] = hat
    return RMSE,R2,Nut_memory

In [325]:
def predict_ts(df,nutrient,model,var_name,hyperparameters):
    values = load_dataset(df,var_name) #values = values.astype('float32')
    # frame as supervised learning
    n_var = len(var_name)
    reframed = reframe(values,hyperparameters,var_name)
    values = reframed.values
    
    # add the predictive values into dataset
    value_X, value_y = values[:, :-1], values[:, -1]
    value_X = value_X.reshape((value_X.shape[0], hyperparameters['time_steps']+1, int(value_X.shape[1]/(hyperparameters['time_steps']+1))))
    y_pred = Scaler.inverse_transform(model.predict(value_X,batch_size = hyperparameters['n_batch']))    
    df[nutrient].iloc[hyperparameters['time_steps']:]=y_pred[:,0]
    df[nutrient].fillna(method = 'backfill',inplace = True)
    return df

In [36]:
# Load testing dataset(Daily data)
def read_daily_df(experiment_time,features,file): # test_time = pd.Timestamp(yyyy,mm,dd)
    os.chdir(trainning_folder)
    Erken_HydMet = pd.read_csv(file,header = 0)
    Erken_HydMet['Date'] = pd.to_datetime(Erken_HydMet['Date'])
    Erken_HydMet = Erken_HydMet[Erken_HydMet['Date']>=experiment_time]
    return Erken_HydMet

In [395]:
def resample(Nut,Erken_Met,sample_interval):
    # Resample the dataset
    drop_row = []
    dT = 0
    i = 1
    Nut_sample = Nut.copy()
    Nut_sample['year'] = Nut_sample['Date'].apply(lambda yr:yr.year)
    Nut_train = Nut_sample[Nut_sample['year']!=test_yr]
    Nut_test = Nut_sample[Nut_sample['year']==test_yr]
    while True:
        dT = (Nut_train['Date'].iloc[i]-Nut_train['Date'].iloc[i-1])/np.timedelta64(1, 'D')
        if dT<sample_interval: # set the data frequency (over 10, 14, 20 days)
            #print(sample_df['Date'].iloc[i])
            Nut_train.drop(Nut_train.index.values[i],axis = 0,inplace  =True)
        else:
            i+=1
        if i>=len(Nut_train):break
    Nut_sample = pd.concat([Nut_train,Nut_test]).sort_values(by= 'Date')
    Nut_memory = Erken_Met.merge(Nut_sample,how = 'left',on = 'Date')
    Nut_memory['year'] = Nut_memory['Date'].apply(lambda d:d.year)
    return Nut_memory

In [374]:
# Load training dataset
os.chdir(trainning_folder)
all_df = pd.read_csv('Process_based_trainning_df.csv',sep = ',')
all_df['Date'] = pd.to_datetime(all_df['Date'])

# Load daily physical factors
file = 'Process_based_Daily_df_with ice therm W.csv'
experiment_time = pd.Timestamp(2004,1,1)
Erken_Met = read_daily_df(experiment_time,features,file) # choose ice data or not

In [375]:
# set the directory
os.chdir(LSTM_save_folder)
# Create the daily df with Nan in nutrients columns
Nut = all_df[['Date','Si(mmole/m3)','TotP(mmole/m3)','NH4(mmole/m3)','NOX(mmole/m3)','PO4(mmole/m3)',
               'O2(mmole/m3)','Chl(mg/m3)']]

In [408]:
# Define the features and years
features = ['Date','month','U','SST','AirT','delT','Humidity','CC','swr(w/m2)','Prec(mm/d)','inflow(m3/s)']
features.extend(['Ice_d','days from iceoff','MLD','W','thermD'])
year = [2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]

In [478]:
Erken_NOX = pd.DataFrame(columns = ['Date','NOX(mmole/m3)'])
Erken_O2 = pd.DataFrame(columns = ['Date','O2(mmole/m3)'])
Erken_NH4 = pd.DataFrame(columns = ['Date','NH4(mmole/m3)'])
Erken_PO4 = pd.DataFrame(columns = ['Date','PO4(mmole/m3)'])
Erken_TP = pd.DataFrame(columns = ['Date','TP(mmole/m3)'])
Erken_Si = pd.DataFrame(columns = ['Date','Si(mmole/m3)'])
Erken_Chl = pd.DataFrame(columns = ['Date','Chl(mg/m3)'])

NOX_RMSE = []
NOX_R2 = []
O2_RMSE = []
O2_R2 = []
NH4_RMSE = []
NH4_R2 = []
PO4_RMSE = []
PO4_R2 = []
TP_RMSE = []
TP_R2 = []
Si_RMSE = []
Si_R2 = []
Chl_RMSE = []
Chl_R2 = []

In [479]:
# set the hyperparameters
hyperparameters = {'n_batch':10,'nb_epoch':100,'n_neuros':200,'dropout':0.1,'time_steps':7,'n_out':1,'verbose':0}

In [480]:
for i in range(len(year)):
    start_time = time.time()
    test_yr = year[i]
    Nut_memory = resample(Nut,Erken_Met,35)
    
    # predict NOX
    nutrient = 'NOX(mmole/m3)'
    print(nutrient)
    var_name = ['delT', 'Ice_d','days from iceoff', 'inflow(m3/s)','U', 'AirT', 'Humidity', 'CC','swr(w/m2)', 
                'Prec(mm/d)', 'MLD','W','thermD',nutrient] # removed 'month' and 'SST' 
    NOX_df,NOX_model,NOX_hat = predict_lstm(Nut_memory,test_yr,var_name,nutrient,hyperparameters)
    # Compare the predictions with observations
    NOX_RMSE,NOX_R2,Nut_memory = compare(NOX_df,Nut_memory,nutrient,NOX_RMSE,NOX_R2,NOX_hat)
    Erken_NOX = pd.concat([Erken_NOX,NOX_df])
    
    ## use the trained model to interplate the whole timeseries
    Nut_memory = predict_ts(Nut_memory,nutrient,NOX_model,var_name,hyperparameters)
    
    # predict O2
    nutrient = 'O2(mmole/m3)'
    print('\n')
    print(nutrient)
    var_name = ['delT', 'Ice_d','days from iceoff', 'inflow(m3/s)','U', 'AirT', 'Humidity', 'CC','swr(w/m2)', 
                'Prec(mm/d)','MLD','W','thermD',nutrient] # removed 'month' and 'SST'
    O2_df,O2_model,O2_hat = predict_lstm(Nut_memory,test_yr,var_name,nutrient,hyperparameters)
    # Compare the predictions with observations
    O2_RMSE,O2_R2,Nut_memory = compare(O2_df,Nut_memory,nutrient,O2_RMSE,O2_R2,O2_hat)
    Erken_O2 = pd.concat([Erken_O2,O2_df])
    ## use the trained model to interplate the whole timeseries
    Nut_memory = predict_ts(Nut_memory,nutrient,O2_model,var_name,hyperparameters)
    
    # predict NH4
    nutrient = 'NH4(mmole/m3)'
    print('\n')
    print(nutrient)
    var_name = ['delT', 'Ice_d','days from iceoff', 'inflow(m3/s)','U', 'AirT', 'Humidity', 'CC','swr(w/m2)', 
                'Prec(mm/d)','NOX(mmole/m3)','O2(mmole/m3)','MLD','W','thermD',nutrient] 
    NH4_df,NH4_model,NH4_hat = predict_lstm(Nut_memory,test_yr,var_name,nutrient,hyperparameters)
    # Compare the predictions with observations
    NH4_RMSE,NH4_R2,Nut_memory = compare(NH4_df,Nut_memory,nutrient,NH4_RMSE,NH4_R2,NH4_hat)
    Erken_NH4 = pd.concat([Erken_NH4,NH4_df])
    ## use the trained model to interplate the whole timeseries
    Nut_memory = predict_ts(Nut_memory,nutrient,NH4_model,var_name,hyperparameters)
    
    # predict PO4
    nutrient = 'PO4(mmole/m3)'
    print('\n')
    print(nutrient)
    var_name = ['delT', 'Ice_d','days from iceoff', 'inflow(m3/s)','U', 'AirT', 'Humidity', 'CC','swr(w/m2)', 
                'Prec(mm/d)','NOX(mmole/m3)','O2(mmole/m3)', 'MLD','W','thermD',nutrient] 
    PO4_df,PO4_model,PO4_hat = predict_lstm(Nut_memory,test_yr,var_name,nutrient,hyperparameters)
    # Compare the predictions with observations
    PO4_RMSE,PO4_R2,Nut_memory = compare(PO4_df,Nut_memory,nutrient,PO4_RMSE,PO4_R2,PO4_hat)
    Erken_PO4 = pd.concat([Erken_PO4,PO4_df])
     ## use the trained model to interplate the whole timeseries
    Nut_memory = predict_ts(Nut_memory,nutrient,PO4_model,var_name,hyperparameters)
   
    # Predict TP
    nutrient = 'TotP(mmole/m3)'
    print('\n')
    print(nutrient)
    var_name = ['delT', 'Ice_d','days from iceoff','inflow(m3/s)','U','AirT','Humidity','CC','swr(w/m2)', 
                'Prec(mm/d)','NOX(mmole/m3)','PO4(mmole/m3)','O2(mmole/m3)','MLD','W','thermD', nutrient] 
    TP_df,TP_model,TP_hat = predict_lstm(Nut_memory,test_yr,var_name,nutrient,hyperparameters)
    # Compare the predictions with observations
    TP_RMSE,TP_R2,Nut_memory = compare(TP_df,Nut_memory,nutrient,TP_RMSE,TP_R2,TP_hat)
    Erken_TP = pd.concat([Erken_TP,TP_df])
    ## use the trained model to interplate the whole timeseries
    Nut_memory = predict_ts(Nut_memory,nutrient,TP_model,var_name,hyperparameters)

    # Predict Si
    nutrient = 'Si(mmole/m3)'
    print('\n')
    print(nutrient)
    var_name = ['delT', 'Ice_d','days from iceoff','inflow(m3/s)','U','AirT','Humidity','CC','swr(w/m2)', 
                'Prec(mm/d)','NOX(mmole/m3)','PO4(mmole/m3)','O2(mmole/m3)','MLD','W','thermD', nutrient] 
    Si_df,Si_model,Si_hat = predict_lstm(Nut_memory,test_yr,var_name,nutrient,hyperparameters)
    # Compare the predictions with observations
    Si_RMSE,Si_R2,Nut_memory = compare(Si_df,Nut_memory,nutrient,Si_RMSE,Si_R2,Si_hat)
    Erken_Si = pd.concat([Erken_Si,Si_df])
    ## use the trained model to interplate the whole timeseries
    Nut_memory = predict_ts(Nut_memory,nutrient,Si_model,var_name,hyperparameters)

    # Predict Chl
    nutrient = 'Chl(mg/m3)'
    print('\n')
    print(nutrient)
    var_name = ['delT', 'Ice_d','days from iceoff','inflow(m3/s)','U','AirT','Humidity','CC','swr(w/m2)','Prec(mm/d)',
                'NOX(mmole/m3)','PO4(mmole/m3)','Si(mmole/m3)', 'TotP(mmole/m3)','NH4(mmole/m3)','O2(mmole/m3)',
                'MLD','W','thermD',nutrient] 
    Chl_df,Chl_model,Chl_hat = predict_lstm(Nut_memory,test_yr,var_name,nutrient,hyperparameters)
    # Compare the predictions with observations
    Chl_RMSE,Chl_R2,Nut_memory = compare(Chl_df,Nut_memory,nutrient,Chl_RMSE,Chl_R2,Chl_hat)
    Erken_Chl = pd.concat([Erken_Chl,Chl_df])
    ## use the trained model to interplate the whole timeseries
    Nut_memory = predict_ts(Nut_memory,nutrient,Chl_model,var_name,hyperparameters)
    
    print(year[i])
    print('Model takes '+str(round((time.time()-start_time)/60))+' min to run')
    print('\n')


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


NOX(mmole/m3)
(3658, 8, 13) (3658,) (195, 8, 13) (195,)
number of input timesteps: 8
number of features: 13
number of samples: 103



A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




O2(mmole/m3)
(3658, 8, 13) (3658,) (195, 8, 13) (195,)
number of input timesteps: 8
number of features: 13
number of samples: 103



A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




Si(mmole/m3)
(3616, 8, 16) (3616,) (237, 8, 16) (237,)
number of input timesteps: 8
number of features: 16
number of samples: 103



A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




Chl(mg/m3)
(3616, 8, 19) (3616,) (237, 8, 19) (237,)
number of input timesteps: 8
number of features: 19
number of samples: 103
2020
Model takes 17 min to run





A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [482]:
Erken_Nut = Erken_NOX.merge(Erken_NH4,
                on='Date',
                how = 'left').merge(Erken_O2,
                                    on='Date',
                                    how = 'left').merge(Erken_PO4,
                                                        on='Date',
                                                        how = 'left').merge(Erken_TP,
                                                                            on = 'Date',
                                                                            how='left').merge(Erken_Si,
                                                                                              on='Date',
                                                                                              how = 'left').merge(Erken_Chl,
                                                                                                                  on='Date',
                                                                                                                  how = 'left')
Erken_Nut.drop('TP(mmole/m3)',axis = 1,inplace = True)
os.chdir(LSTM_save_folder)
Erken_Nut.to_csv('LSTM predicted nutrient_predicted nutrient and Chl.csv',index = False)

RMSE = pd.DataFrame({'NOX':NOX_RMSE,'NH4':NH4_RMSE,'O2':O2_RMSE,'PO4':PO4_RMSE,'TP':TP_RMSE,'Si':Si_RMSE,'Chl':Chl_RMSE},index = year)
RMSE.to_csv('RMSE.csv',index = False)

R2 = pd.DataFrame({'NOX':NOX_R2,'NH4':NH4_R2,'O2':O2_R2,'PO4':PO4_R2,'TP':TP_R2,'Si':Si_R2,'Chl':Chl_R2},index = year)
R2.to_csv('R2.csv',index = False)

In [481]:
Chl_RMSE

[4.442661438938522,
 3.1676660626627036,
 5.336711898160277,
 4.986615127780393,
 3.8253900878157414,
 3.9707920631430946,
 4.846255741082887,
 5.376003035349939,
 3.6447674853650867,
 8.630015809654916,
 5.967081769372331,
 5.064256693092051,
 4.579564181484866,
 4.26212376823703,
 5.895335111002262,
 6.485282607743182,
 4.527411965693013]